In [1]:
import pandas as pd
import numpy as np

In [2]:
%run chart_theme.py

In [3]:
nat1 = pd.read_csv('national_rent_vs_mort_and_inc_1yr_1018.csv')
nat5 = pd.read_csv('national_rent_vs_mort_and_inc_5yr_0816.csv')

### National Med. Housing % of Income

In [4]:
# Supplementing 1-yr data w/ 08 & 09 from 5-yr
nat_comb = pd.concat([nat5[nat5.Year < 2010], nat1])

In [80]:
nat_mort = alt.Chart(nat_comb
                    ).mark_line(size=7, strokeCap='round', color='#97c2f2', interpolate='basis'
                               ).encode(x=alt.X('Year:O', axis=alt.Axis(values=[2008,2018], ticks=True, tickSize=13, 
                                                                        labelFontSize=22, labelOpacity=.7, labelPadding=13)), 
                                        y=alt.Y('Mort_Share_Inc_whole',
                                               axis=alt.Axis(values=list(range(15,40,5)), 
                                                             labelPadding=30, 
                                                             labelOpacity=.7,
                                                             labelFontSize=22), 
                                               scale=alt.Scale(domain=[10,40])))

In [81]:
nat_rent = alt.Chart(nat_comb, title='Income Devoted to Housing'
                    ).mark_line(size=5, strokeCap='round', color='#cede54', interpolate='basis'
                               ).encode(x=alt.X('Year:O', axis=alt.Axis(values=[2008,2018], ticks=True, tickSize=13, 
                                                                        labelFontSize=22, labelOpacity=.7, labelPadding=13)), 
                                        y=alt.Y('Rent_Share_Inc_whole', title='Relative housing expenses have trended downwards since 2010',
                                               axis=alt.Axis(values=list(range(15,40,5)), 
                                                             titleX=-33, titleY=-40, titleOpacity=.7, titleFontSize=26,
                                                             labelPadding=30, labelOpacity=.7, labelFontSize=22),
                                               scale=alt.Scale(domain=[10,40])))

In [95]:
nat_mort_text_df = pd.DataFrame({'Year':(range(2008, 2011)), 
                                 'Numbers':[0,0,40], 
                                 'Text':['Renters', 'Homeowners', '_'*76]})

In [102]:
renters = alt.Chart(nat_mort_text_df.query('Year == 2008')).mark_text(dx=608, dy=-418, font='lato', fontSize=30, color='#b7c83e').encode(x='Year:O', y='Numbers', text='Text')
homeowners = alt.Chart(nat_mort_text_df.query('Year == 2009')).mark_text(dx=590, dy=-225, font='lato', fontSize=30, color='#81addc').encode(x='Year:O', y='Numbers', text='Text')
line = alt.Chart(nat_mort_text_df.query('Year == 2010')).mark_text(dx=222, dy=-4, opacity=.09, fontSize=11.5, fontWeight='bold').encode(x='Year:O', y='Numbers', text='Text')
y_labels_df = pd.DataFrame({'x':[2008], 'y':[40], 'text':['40% of income']})
y_labels = alt.Chart(y_labels_df.query('x == 2008')).mark_text(dx=12, dy=0, font='lato', fontSize=22, opacity=.7).encode(x='x:O', y='y', text='text')

In [106]:
nat_comb_plot = (nat_mort + nat_rent + renters + homeowners + y_labels + line).configure_title(fontSize=32)

In [107]:
nat_comb_plot

alt.LayerChart(...)

In [108]:
nat_comb_plot.save('nat_rent_mort_per_inc.svg')

---

### Change in Housing Expenses:

In [4]:
rent_mort_inc5 = pd.read_csv('national_rent_vs_mort_and_inc_5yr_0816.csv')
rent_mort_inc1 = pd.read_csv('national_rent_vs_mort_and_inc_1yr_1018.csv')

In [5]:
vals = []
cols = ['Med_Mort', 'Med_Rent', 'Med_Mort_Inc', 'Med_Rent_Inc']
for col in cols:
    vals.append([i*100 for i in rent_mort_inc1[col].pct_change().cumsum()])

In [6]:
pct_chg_df = pd.DataFrame(vals).transpose()
pct_chg_df.columns = cols
pct_chg_df['Year'] = list(range(2010, 2019))
pct_chg_df = pct_chg_df.fillna(0)

In [7]:
pct_chg_df = pct_chg_df.melt(id_vars='Year', var_name='Value', value_name='Pct_Change')

In [8]:
# Medium article formatting (label size, etc.)
rh_housing1 = alt.Chart(pct_chg_df.iloc[:9], title='Change in Housing Expenses and Income'
         ).mark_area(interpolate='basis', color='#81addc', opacity=1
         ).encode(x=alt.X('Year:O', axis=alt.Axis(values=[2010, 2018], ticks=True, tickSize=10, labelFontSize=18, labelOpacity=.7)), 
                  y=alt.Y('Pct_Change', title= 'Income growth outpaced housing expenses for both renters and homeowners', 
                          axis=alt.Axis(values=list(range(-5,30,5)), 
                                        titleFontSize=20, titleOpacity=.7, titleX=-25, titleY=-30,
                                        labelPadding = 25, labelFontSize=18, labelOpacity=.7),
                          scale=alt.Scale(domain=[-10,30])))

rh_housing2 = alt.Chart(pct_chg_df.iloc[9:18],
         ).mark_area(interpolate='basis', color='#b7c83e', opacity=1
         ).encode(x='Year:O', y=alt.Y('Pct_Change'))

rh_income1 = alt.Chart(pct_chg_df.iloc[18:27],
         ).mark_line(size=6, strokeCap='round', interpolate='basis', color='#81addc'
         ).encode(x='Year:O', y=alt.Y('Pct_Change'))

rh_income2 = alt.Chart(pct_chg_df.iloc[27:],
         ).mark_line(size=6, strokeCap='round', interpolate='basis', color='#b7c83e'
         ).encode(x='Year:O', y=alt.Y('Pct_Change'))

df = pd.DataFrame({'x':list(range(2010,2016)), 'y':[30,30,29.3,27.3,22.9,20.9], 'text':['30% increase', '_'*75, 'Renter', 'Homeowner', 'Income', 'Expenses']})
text = alt.Chart(df.query('x==2010')).mark_text(dx=-5, font='lato', fontSize=18, opacity=.7).encode(x='x:O', y='y', text='text')
line = alt.Chart(df.query('x==2011')).mark_text(dx=230, dy=-4, opacity=.09, fontSize=11.5, fontWeight='bold').encode(x='x:O', y='y', text='text')
rent_inc1 = alt.Chart(df.query('x==2012')).mark_text(dx=446, dy=-5, font='lato', fontSize=24, color='#b7c83e').encode(x='x:O', y='y', text='text')
rent_inc2 = alt.Chart(df.query('x==2014')).mark_text(dx=316, dy=-48, font='lato', fontSize=24, color='#b7c83e').encode(x='x:O', y='y', text='text')
home_inc1 = alt.Chart(df.query('x==2013')).mark_text(dx=410, dy=42, font='lato', fontSize=24, color='#81addc').encode(x='x:O', y='y', text='text')
home_inc2 = alt.Chart(df.query('x==2014')).mark_text(dx=316, dy=20, font='lato', fontSize=24, color='#81addc').encode(x='x:O', y='y', text='text')
rent_exp1 = alt.Chart(df.query('x==2012')).mark_text(dx=272, dy=215, font='lato', fontSize=22, fontWeight='bold', color='white').encode(x='x:O', y='y', text='text')
rent_exp2 = alt.Chart(df.query('x==2015')).mark_text(dx=85, dy=150, font='lato', fontSize=22, fontWeight='bold', color='white').encode(x='x:O', y='y', text='text')
home_exp1 = alt.Chart(df.query('x==2013')).mark_text(dx=37, dy=290, font='lato', fontSize=22, fontWeight='bold', color='white').encode(x='x:O', y='y', text='text')
home_exp2 = alt.Chart(df.query('x==2015')).mark_text(dx=-111, dy=247, font='lato', fontSize=22, fontWeight='bold', color='white').encode(x='x:O', y='y', text='text')

In [59]:
# Website formatting (label size, etc.)
rh_housing1 = alt.Chart(pct_chg_df.iloc[:9], title='Change in Housing Expenses and Income'
         ).mark_area(interpolate='basis', color='#81addc', opacity=1
         ).encode(x=alt.X('Year:O', axis=alt.Axis(values=[2010, 2018], ticks=True, tickSize=10, labelFontSize=18, labelOpacity=.7)), 
                  y=alt.Y('Pct_Change', title= 'Income growth outpaced housing expenses for both renters and homeowners', 
                          axis=alt.Axis(values=list(range(-5,30,5)), 
                                        titleFontSize=20, titleOpacity=.7, titleX=-25, titleY=-30,
                                        labelPadding = 25, labelFontSize=18, labelOpacity=.7),
                          scale=alt.Scale(domain=[-10,30])))

rh_housing2 = alt.Chart(pct_chg_df.iloc[9:18],
         ).mark_area(interpolate='basis', color='#b7c83e', opacity=1
         ).encode(x='Year:O', y=alt.Y('Pct_Change'))

rh_income1 = alt.Chart(pct_chg_df.iloc[18:27],
         ).mark_line(size=6, strokeCap='round', interpolate='basis', color='#81addc'
         ).encode(x='Year:O', y=alt.Y('Pct_Change'))

rh_income2 = alt.Chart(pct_chg_df.iloc[27:],
         ).mark_line(size=6, strokeCap='round', interpolate='basis', color='#b7c83e'
         ).encode(x='Year:O', y=alt.Y('Pct_Change'))

df = pd.DataFrame({'x':list(range(2010,2016)), 'y':[30,30,29.3,27.3,22.9,20.9], 'text':['30% increase', '_'*76, 'Renter', 'Homeowner', 'Income', 'Expenses']})
text = alt.Chart(df.query('x==2010')).mark_text(dx=-5, font='lato', fontSize=18, opacity=.7).encode(x='x:O', y='y', text='text')
line = alt.Chart(df.query('x==2011')).mark_text(dx=230, dy=-4, opacity=.09, fontSize=11.5, fontWeight='bold').encode(x='x:O', y='y', text='text')
rent_inc1 = alt.Chart(df.query('x==2012')).mark_text(dx=445.5, dy=-5, font='lato', fontSize=22, color='#b7c83e').encode(x='x:O', y='y', text='text')
rent_inc2 = alt.Chart(df.query('x==2014')).mark_text(dx=315, dy=-48, font='lato', fontSize=22, color='#b7c83e').encode(x='x:O', y='y', text='text')
home_inc1 = alt.Chart(df.query('x==2013')).mark_text(dx=406.5, dy=42, font='lato', fontSize=22, color='#81addc').encode(x='x:O', y='y', text='text')
home_inc2 = alt.Chart(df.query('x==2014')).mark_text(dx=315, dy=20, font='lato', fontSize=22, color='#81addc').encode(x='x:O', y='y', text='text')
rent_exp1 = alt.Chart(df.query('x==2012')).mark_text(dx=272, dy=215, font='lato', fontSize=22, fontWeight='bold', color='white').encode(x='x:O', y='y', text='text')
rent_exp2 = alt.Chart(df.query('x==2015')).mark_text(dx=85, dy=150, font='lato', fontSize=22, fontWeight='bold', color='white').encode(x='x:O', y='y', text='text')
home_exp1 = alt.Chart(df.query('x==2013')).mark_text(dx=37, dy=290, font='lato', fontSize=22, fontWeight='bold', color='white').encode(x='x:O', y='y', text='text')
home_exp2 = alt.Chart(df.query('x==2015')).mark_text(dx=-111, dy=247, font='lato', fontSize=22, fontWeight='bold', color='white').encode(x='x:O', y='y', text='text')

In [60]:
inc_hous_chg = (rh_housing1 + rh_housing2 + rh_income1 + rh_income2 + text + line + rent_inc1 + rent_inc2 + home_inc1 + home_inc2 + rent_exp1 + rent_exp2 + home_exp1 + home_exp2).configure_title(fontSize=28)

In [61]:
inc_hous_chg

alt.LayerChart(...)

In [62]:
inc_hous_chg.save('inc_hous_chg_1018.svg')

---

In [252]:
nat_mort_inc = alt.Chart(rent_mort_inc1
                    ).mark_area(opacity=.7, strokeCap='round', color='#97c2f2', interpolate='basis'
                               ).encode(x=alt.X('Year:O', axis=alt.Axis(values=[2010,2018], ticks=True, tickSize=13, 
                                                                        labelFontSize=23, labelOpacity=.7, labelPadding=13)), 
                                        y=alt.Y('Med_Mort_Inc',
                                               axis=alt.Axis(values=list(range(1000, 8000, 1000)), format='~s', 
                                                             labelPadding=30, 
                                                             labelOpacity=.7,
                                                             labelFontSize=23)))

In [253]:
nat_rent_inc = alt.Chart(rent_mort_inc1, title='Median Income (Renters)'
                    ).mark_area(opacity=.7, strokeCap='round', color='#cede54', interpolate='basis'
                               ).encode(x=alt.X('Year:O', axis=alt.Axis(values=[2010,2018], ticks=True, tickSize=13, labelFontSize=23, 
                                                                        labelOpacity=.7, labelPadding=13)), 
                                        y=alt.Y('Med_Rent_Inc',
                                               axis=alt.Axis(values=list(range(1000, 8000, 1000)), format='~s', 
                                                             titleX=-31, titleY=-40, titleOpacity=.7, titleFontSize=26,
                                                             labelPadding=30, labelOpacity=.7, labelFontSize=23)))

In [254]:
nat_mort = alt.Chart(rent_mort_inc1
                    ).mark_line(size=5, strokeCap='round', color='#97c2f2', interpolate='basis'
                               ).encode(x=alt.X('Year:O', axis=alt.Axis(values=[2010,2018], ticks=True, tickSize=13, 
                                                                        labelFontSize=16, labelOpacity=.7, labelPadding=13)), 
                                        y=alt.Y('Med_Mort',
                                               axis=alt.Axis(labelPadding=40, labelOpacity=.7,labelFontSize=16, format='<a'),
                                               scale=alt.Scale(domain=[400,1400])))

In [268]:
nat_rent = alt.Chart(rent_mort_inc1, title='Monthly Housing Expenses'
                    ).mark_line(size=5, strokeCap='round', color='#cede54', interpolate='basis'
                               ).encode(x=alt.X('Year:O', axis=alt.Axis(values=[2010,2018], ticks=True, tickSize=13, 
                                                                        labelFontSize=16, labelOpacity=.7, labelPadding=13)), 
                                        y=alt.Y('Med_Rent', title='Renter housing expenses are now almost equal to homeowner expenses',
                                               axis=alt.Axis(values=list(range(600, 1300, 200)), format='<a',
                                                             titleX=-40, titleY=-40, titleOpacity=.7, titleFontSize=18,
                                                             labelPadding=40, labelOpacity=.7,labelFontSize=16),
                                               scale=alt.Scale(domain=[400,1400])))

df = pd.DataFrame({'x':[2010,2011,2017,2016], 'y':[1400, 1400, 1150, 930], 'text':['$1400 per month', '_'*81, 'Homeowners', 'Renters']})
y_label = alt.Chart(df.query('x==2010')
                   ).mark_text(dx=-12, font='lato', fontSize=16, opacity=.7
                   ).encode(x='x:O', y='y', text='text')

line = alt.Chart(df.query('x==2011')).mark_text(dx=242, dy=-4, opacity=.09, 
                                                fontSize=11.5, fontWeight='bold').encode(x='x:O', y='y', text='text')

homeowners = alt.Chart(df.query('x==2017')).mark_text(dx=0, dy=0, font='lato', fontSize=22, color='#81addc').encode(x='x:O', y='y', text='text')
renters = alt.Chart(df.query('x==2016')).mark_text(dx=60, dy=0, font='lato', fontSize=22, color='#b7c83e').encode(x='x:O', y='y', text='text')

In [269]:
hous_expens_chart = (nat_mort + nat_rent + y_label + line + homeowners + renters).configure_title(fontSize=22)
hous_expens_chart

alt.LayerChart(...)

In [267]:
hous_expens_chart.save('mort_rent_exp_1018.svg')

---